# Core ML: Bayesian NNs via MCMC


## Import libraries

In [1]:
import numpy as np
from pathlib import Path


## Prepare noisy XOR dataset

In [2]:
# Define function for simulating noisy XOR points

def simulate_xor(n=np.repeat(100, 4), s=0.55):
	x = np.vstack([
            # points corresponding to (0, 0)
            np.column_stack(
            	[np.random.rand(n[0]) - s, np.random.rand(n[0]) - s]),
            # points corresponding to (0, 1)
            np.column_stack(
            	[np.random.rand(n[1]) - s, np.random.rand(n[1]) + s]),
            # points corresponding to (1, 0)
            np.column_stack(
            	[np.random.rand(n[2]) + s, np.random.rand(n[2]) - s]),
            # points corresponding to (1, 1)
            np.column_stack(
            	[np.random.rand(n[3]) + s, np.random.rand(n[3]) + s])
	])

	y = np.repeat([0, 1, 1, 0], n)

	return x, y


In [7]:
# Initialize constants for dataset

num_classes = 4
num_training_samples = np.repeat(1200, num_classes)
num_test_samples = np.repeat(300, num_classes)

In [8]:
# Create train and test dataset

x_train, y_train = simulate_xor(n=num_training_samples)
x_test, y_test = simulate_xor(n=num_test_samples)

In [9]:
# Save train dataset to file

train_data_path = Path("data/train")

train_data_path.mkdir(parents=True, exist_ok=True)

np.savetxt(train_data_path.joinpath("x_train.csv"), x_train, delimiter=",", header="x1,x2", comments="")
np.savetxt(train_data_path.joinpath("y_train.csv"), y_train,
           delimiter=",", header="y", comments="")


In [10]:
# Save test dataset to file

test_data_path = Path("data/test")

test_data_path.mkdir(parents=True, exist_ok=True)

np.savetxt(test_data_path.joinpath("x_test.csv"), x_test,
           delimiter=",", header="x1,x2", comments="")
np.savetxt(test_data_path.joinpath("y_test.csv"), y_test,
           delimiter=",", header="y", comments="")
